# Bibliothèque

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt
from scipy.io import  wavfile as wav
from playsound import playsound as play  
from IPython.display import display, Audio

# Lecture des fichier sonore

In [26]:
Fe, signal = wav.read("saxo.wav")
display(Audio(signal, rate=Fe))


882

# Découpage du signal en trame

In [29]:
Te = 1/Fe
N = len(signal)
# Calcul du nombre d'echantillons par trame de 20 ms
n_trame = int(20*10**(-3) / Te)
nb_trame = int(N/n)

In [30]:
signal_trame = np.zeros((nb_trame, n_trame))
for i in range(nb_trame):
    signal_trame [i, :] = signal[i*n_trame:(i+1)*n_trame]
    

# Fenêtrage

In [36]:
H = np.hamming(n_trame)
signal_fenetre = np.zeros(signal_trame.shape)
signal_fenetre = signal_trame*H 

# Fonction pour découper en trame et fenetrer un signal

In [64]:
def dec_trame(signal, Fe, temps_trame):
    Te = 1/Fe
    N = len(signal)
    
    # Calcul du nombre d'echantillons par trame de 20 ms
    n_trame = int(temps_trame / Te)
    nb_trame = int(N/n)
    
    # Découage  en trame de temps_trame
    signal_trame = np.zeros((nb_trame, n_trame))
    for i in range(nb_trame):
        signal_trame [i, :] = signal[i*n_trame:(i+1)*n_trame]
    
    # Fenetrage
    x_fenetre = signal_trame * np.hamming(n_trame)

    return  x_fenetre 

In [65]:
sf = dec_trame(signal, Fe, 20*10**(-3))

# Construire la Matrice $R_{pk}$

In [190]:
K = 4
Rpk = np.zeros((nb_trame,2*K, 2*K))
Rp = np.zeros((nb_trame, 2*K))
ak = np.zeros((nb_trame, 2*K))
cpt=0
for i in range(len(signal_trame)):
    for p in range(2*K):
        for k in range(2*K):
            for e in range(len(signal_trame[i])):
                if (e-p) >= 0 and (e-k)>= 0:
                    Rpk[i,p,k] += signal_trame[i, e-p] *signal_trame[i, e-k]
                else:
                    Rpk[i,p,k] = 0
                     
    cpt+=1

In [191]:
for i in range(len(Rpk)):
    for p in range(8):
        for e in range(np.abs(p), len(signal_trame[i])):
            if e-p >= 0:
                Rp[i,p] += signal_trame[i, e-p] *signal_trame[i, e]
            else:
                Rp[i,p] = 0
                
                

In [219]:
index = 100
ak[index] = np.dot(np.linalg.inv(Rpk[index]),(-Rp[index]))
ak[index]

array([-1.00000000e+00, -1.13686838e-13,  3.41060513e-13, -1.98951966e-13,
        2.84217094e-14, -4.26325641e-14,  2.13162821e-14, -1.06581410e-14])

In [173]:
ak[i]

array([0., 0., 0., 0., 0., 0., 0., 0.])